In [1]:
import pandas as pd 
import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import os
from pathlib import Path 

In [2]:
nont = '/home/ruijingyang/cytotox/eda_phase/data/train_data_concentrateA_norm_m3/non-toxic'
tox = '/home/ruijingyang/cytotox/eda_phase/data/train_data_concentrateA_norm_m3/toxic'

In [4]:
len(os.listdir(nont))

12

In [5]:
len(os.listdir(tox))

28

In [2]:
cur_dir = os.getcwd()
data_src = os.path.join(cur_dir, 'data', 'Cytotox_HCA_065_20230907_B0_TIFFs')
data_src

'/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs'

In [3]:
image_list = os.listdir(data_src)
image_path_list = [os.path.join(data_src, f) for f in image_list]
len(image_path_list)

3840

In [18]:
int(Path(image_path_list[0]).stem.rsplit('-', 1)[1][-1])

5

### Merge 5 channels to 1

In [21]:
outpath = '/home/ruijingyang/cytotox/eda_phase/data/norm_merged5'

In [22]:
def normalize_channel(image_array, min_value, max_value):
    normalized_array = (image_array - min_value) / (max_value - min_value)
    return normalized_array

In [23]:
channel_dict = {
    1: [179, 65535],
    2: [345, 65535],
    3: [0, 38383],
    4: [144, 10705],
    5: [3621, 65535]}

In [19]:
# 组织图像文件
grouped_files = {}
for file in image_path_list:
    filename = Path(file).stem
    base_name = filename.rsplit('-', 1)[0]
    channel = int(filename.rsplit('-', 1)[1][-1]) #get the last ind
    if base_name not in grouped_files:
        grouped_files[base_name] = [None] * 5  # 创建一个长度为5的空列表
    grouped_files[base_name][channel-1] = file  # 根据channel数字，放置在正确的位置


In [20]:
grouped_files

{'006023-1': ['/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/006023-1-001001001.tif',
  '/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/006023-1-001001002.tif',
  '/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/006023-1-001001003.tif',
  '/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/006023-1-001001004.tif',
  '/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/006023-1-001001005.tif'],
 '014021-2': ['/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/014021-2-001001001.tif',
  '/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/014021-2-001001002.tif',
  '/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/014021-2-001001003.tif',
  '/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/014021-2-001001004.tif',
  '/home/ruijingyang/cytotox/eda_phase/data/Cyto

In [25]:
grouped_files.items()

dict_items([('006023-1', ['/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/006023-1-001001001.tif', '/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/006023-1-001001002.tif', '/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/006023-1-001001003.tif', '/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/006023-1-001001004.tif', '/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/006023-1-001001005.tif']), ('014021-2', ['/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/014021-2-001001001.tif', '/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/014021-2-001001002.tif', '/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/014021-2-001001003.tif', '/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/014021-2-001001004.tif', '/home/ruijingyang/cytotox/eda_phase/data/Cytotox

### Combine and save 5 channels

In [29]:
# 读取、合并、保存图像
for base_name, group in grouped_files.items():
    print(base_name)
    normalized_arrays = []
    for file in group:
        print(file)
        if file:  # 检查是否为None
            ch = Path(file).stem[-1]
            if int(ch) in channel_dict.keys():
                print(ch)
                minmax_list = channel_dict[int(ch)]
                im_arr = np.array(Image.open(file))
                print(im_arr.shape)
                norm_arr = normalize_channel(im_arr, minmax_list[0], minmax_list[1])
                norm_arr_unit8 = np.uint8(norm_arr * 255)
                normalized_arrays.append(norm_arr_unit8)
        else:
            print(f"missing channel image at {base_name} in {group}")
            
    # 假设所有图像大小相同，使用np.dstack将它们沿深度方向堆叠起来
    combined = np.dstack(normalized_arrays)
    
    # note: to save 5 channel tif
    save_imge_path = os.path.join(outpath, f'{base_name}_norm_combined.tif')
    combined_img = Image.fromarray(combined, 'RGB').save(save_imge_path)
    


006023-1
/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/006023-1-001001001.tif
1
(1080, 1080)
/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/006023-1-001001002.tif
2
(1080, 1080)
/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/006023-1-001001003.tif
3
(1080, 1080)
/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/006023-1-001001004.tif
4
(1080, 1080)
/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/006023-1-001001005.tif
5
(1080, 1080)
014021-2
/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/014021-2-001001001.tif
1
(1080, 1080)
/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/014021-2-001001002.tif
2
(1080, 1080)
/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/014021-2-001001003.tif
3
(1080, 1080)
/home/ruijingyang/cytotox/eda_phase/data/Cytotox_HCA_065_20230907_B0_TIFFs/014

### combine and save 3 channels

In [30]:
outpath3 = '/home/ruijingyang/cytotox/eda_phase/data/norm_merged3'

In [31]:
selected_channels = [1,3,4]

In [32]:
# 读取、合并、保存图像
for base_name, group in grouped_files.items():
    print(base_name)
    normalized_arrays = []
    for file in group:
        # print(file)
        if file:  # 检查是否为None
            ch = Path(file).stem[-1]
            if int(ch) in channel_dict.keys() and int(ch) in selected_channels:
                print(ch)
                minmax_list = channel_dict[int(ch)]
                im_arr = np.array(Image.open(file))
                # print(im_arr.shape)
                norm_arr = normalize_channel(im_arr, minmax_list[0], minmax_list[1])
                norm_arr_unit8 = np.uint8(norm_arr * 255)
                normalized_arrays.append(norm_arr_unit8)
        else:
            print(f"missing channel image at {base_name} in {group}")
            
    # 假设所有图像大小相同，使用np.dstack将它们沿深度方向堆叠起来
    combined = np.dstack(normalized_arrays)
    
    # note: to save 5 channel tif
    save_imge_path = os.path.join(outpath3, f'{base_name}_norm_combined.tif')
    combined_img = Image.fromarray(combined, 'RGB').save(save_imge_path)
    


006023-1
1
3
4
014021-2
1
3
4
014004-1
1
3
4
001018-2
1
3
4
004005-2
1
3
4
008020-2
1
3
4
015010-2
1
3
4
013018-1
1
3
4
001005-1
1
3
4
012023-1
1
3
4
006022-2
1
3
4
007001-1
1
3
4
010005-2
1
3
4
005002-2
1
3
4
004008-2
1
3
4
004013-2
1
3
4
015015-1
1
3
4
010008-2
1
3
4
008007-1
1
3
4
003018-2
1
3
4
011023-1
1
3
4
007022-2
1
3
4
010019-1
1
3
4
013021-1
1
3
4
002002-2
1
3
4
008022-2
1
3
4
015012-1
1
3
4
001003-2
1
3
4
003005-1
1
3
4
015015-2
1
3
4
014023-2
1
3
4
002023-1
1
3
4
009004-1
1
3
4
010009-1
1
3
4
003003-2
1
3
4
005013-1
1
3
4
004015-1
1
3
4
011011-1
1
3
4
010022-1
1
3
4
004016-2
1
3
4
003014-1
1
3
4
011014-2
1
3
4
005021-2
1
3
4
003013-1
1
3
4
016003-2
1
3
4
013006-1
1
3
4
015003-2
1
3
4
001024-2
1
3
4
006009-1
1
3
4
009018-2
1
3
4
002005-1
1
3
4
002021-2
1
3
4
015002-1
1
3
4
010005-1
1
3
4
001015-2
1
3
4
014014-1
1
3
4
003008-2
1
3
4
012024-2
1
3
4
002018-1
1
3
4
016014-1
1
3
4
009007-1
1
3
4
010018-2
1
3
4
008016-1
1
3
4
011022-1
1
3
4
003017-2
1
3
4
004015-2
1
3
4
006008-2
1

### Get min max of each channel

In [20]:
channel = 1
selected_paths = [path for path in image_path_list if path.endswith(f"{channel}.tif")]

In [26]:
len(selected_paths)

768

In [28]:
channel_dict = {}
def get_channel_min_max(channel, image_path_list):
    selected_paths = [path for path in image_path_list if path.endswith(f"{channel}.tif")]
    min_chn1, max_chn1 = 1000000, 0
    for image_path in selected_paths:
        image_array = np.array(Image.open(image_path))
        assert image_array.shape == (1080, 1080)
        if image_array.min() <= min_chn1:
            min_chn1 = image_array.min()
        if image_array.max() >= max_chn1:
            max_chn1 = image_array.max()
    channel_dict[channel] = [min_chn1, max_chn1]
    return channel_dict
    

In [29]:
def normalize_channel(image_array, min_value, max_value):
    normalized_array = (image_array - min_value) / (max_value - min_value)
    return normalized_array

In [30]:
ch1 = get_channel_min_max(1, image_path_list)
ch1

{1: [179, 65535]}

In [31]:
ch2 = get_channel_min_max(2, image_path_list)
ch2

{1: [179, 65535], 2: [345, 65535]}

In [32]:
ch3 = get_channel_min_max(3, image_path_list)
ch3

{1: [179, 65535], 2: [345, 65535], 3: [0, 38383]}

In [33]:
ch4 = get_channel_min_max(4, image_path_list)
ch4

{1: [179, 65535], 2: [345, 65535], 3: [0, 38383], 4: [144, 10705]}

In [34]:
ch5 = get_channel_min_max(5, image_path_list)
ch5

{1: [179, 65535],
 2: [345, 65535],
 3: [0, 38383],
 4: [144, 10705],
 5: [3621, 65535]}

In [35]:
small_set_path = '/home/ruijingyang/cytotox/eda_phase/data/sample1'
small_set_list = os.listdir(small_set_path)

In [53]:
normalized_arrays = []
for image in small_set_list:
    ch = Path(image).stem[-1]
    if int(ch) in channel_dict.keys():
        minmax_list = channel_dict[int(ch)]
        im_arr = np.array(Image.open(os.path.join(small_set_path, image)))
        norm_arr = normalize_channel(im_arr, minmax_list[0], minmax_list[1])
        norm_arr_unit8 = np.uint8(norm_arr * 255)
        normalized_arrays.append(norm_arr_unit8)
    else:
        continue

In [56]:
len(normalized_arrays)

3

In [57]:
rgb_image_array = np.dstack(normalized_arrays)

In [58]:
rgb_image = Image.fromarray(rgb_image_array)

In [59]:
rgb_image.save('normalized_merged_image.tif')

### TODO: merge the following

In [ ]:
selected_channels = [1, 3, 4]#['DRAQ7', 'Mitotracker Red', 'Hoechst (nuclei label) ']